In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
from PIL import Image

# Function to check if a file is an image
def is_image_file(filename):
    try:
        img = Image.open(filename)
        img.verify()  # Verify that it is an image
        return True
    except (IOError, SyntaxError):
        return False

# Function to filter invalid images
def filter_invalid_images(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            filepath = os.path.join(root, file)
            if not is_image_file(filepath):
                print(f'Removing invalid image: {filepath}')
                os.remove(filepath)  # Remove invalid image

# Define directories
train_dir = 'C:/Users/Badari/Downloads/PCOS detection using ultrasound images/data/train'
test_dir = 'C:/Users/Badari/Downloads/PCOS detection using ultrasound images/data/test'

# Filter invalid images from dataset
filter_invalid_images(train_dir)
filter_invalid_images(test_dir)

# Image preprocessing and augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Image preprocessing for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

# Load the pre-trained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model layers
base_model.trainable = False

# Create a new model on top of the pre-trained base
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and model checkpointing
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model with specified parameters
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[model_checkpoint, early_stopping]
)

# Load the saved model for evaluation
saved_model = tf.keras.models.load_model('best_model.keras')

# Evaluate the saved model on the test set
loss, accuracy = saved_model.evaluate(test_generator)
print(f'Saved Model Test Accuracy: {accuracy * 100:.2f}%')


Found 1924 images belonging to 2 classes.
Found 1922 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 45s 660ms/step - accuracy: 0.8622 - loss: 1.1901 - val_accuracy: 1.0000 - val_loss: 1.4213e-04
Epoch 2/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 35s 560ms/step - accuracy: 0.9980 - loss: 0.0097 - val_accuracy: 1.0000 - val_loss: 1.7780e-09
Epoch 3/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 36s 575ms/step - accuracy: 0.9933 - loss: 0.0271 - val_accuracy: 1.0000 - val_loss: 1.0883e-07
Epoch 4/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 36s 573ms/step - accuracy: 1.0000 - loss: 2.9516e-04 - val_accuracy: 1.0000 - val_loss: 3.6378e-06
Epoch 5/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 36s 572ms/step - accuracy: 0.9995 - loss: 7.8591e-04 - val_accuracy: 1.0000 - val_loss: 4.7007e-04
Epoch 6/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 36s 574ms/step - accuracy: 0.9937 - loss: 0.0199 - val_accuracy: 1.0000 - val_loss: 2.2614e-08
Epoch 7/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 37s 586ms/step - accura

C:\Users\Badari\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 322 variables whereas the saved optimizer has 10 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


61/61 ━━━━━━━━━━━━━━━━━━━━ 17s 255ms/step - accuracy: 1.0000 - loss: 1.4128e-04
Saved Model Test Accuracy: 100.00%


In [11]:
model.save('C:/Users/Badari/OneDrive/Desktop/projects/girls/final_model.keras')

In [12]:
# Load the saved model for evaluation
saved_model = tf.keras.models.load_model('best_model.keras')

# Evaluate the saved model on the test set
loss, accuracy = saved_model.evaluate(test_generator)
print(f'Saved Model Test Accuracy: {accuracy * 100:.2f}%')

61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 273ms/step - accuracy: 1.0000 - loss: 8.9076e-05
Saved Model Test Accuracy: 100.00%
